In [1]:
!wget https://github.com/boedybios/kaggle_explorations/raw/master/Iris_Data_Classification_and_EDA/dataset/iris/Iris.csv

--2021-09-28 06:15:59--  https://github.com/boedybios/kaggle_explorations/raw/master/Iris_Data_Classification_and_EDA/dataset/iris/Iris.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/boedybios/kaggle_explorations/master/Iris_Data_Classification_and_EDA/dataset/iris/Iris.csv [following]
--2021-09-28 06:15:59--  https://raw.githubusercontent.com/boedybios/kaggle_explorations/master/Iris_Data_Classification_and_EDA/dataset/iris/Iris.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5107 (5.0K) [text/plain]
Saving to: ‘Iris.csv’

Iris.csv            100%[===================>]   4.99K  --.-KB/

In [2]:
#mengimport numpy, pandas dan scipy
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
#membaca dataset dari file ke pandas dataFrame
iris_Dataset = pd.read_csv('/content/Iris.csv',
                          delimiter=',', header=0)

In [5]:
iris_Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [4]:
iris_Dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
#mengubah kelas (kolom "Species") dari string ke unique-integer
iris_Dataset["Species"] = pd.factorize(iris_Dataset.Species)[0]
#menghapus kolom "Id"
iris_Dataset = iris_Dataset.drop(labels="Id", axis=1)

In [7]:
iris_Dataset.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [10]:
att = iris_Dataset[["SepalLengthCm", "SepalWidthCm","PetalLengthCm","PetalWidthCm"]].values

In [12]:
#mengubah dataFrame ke array numpy
irisDataset = iris_Dataset.to_numpy()

In [13]:
#membagi dataset, 40 baris data untuk training
#dan 20 baris data untuk testing
dataTraining = np.concatenate((irisDataset[0:40,:],
                               irisDataset[50:90,:]),axis =0)
dataTetsing = np.concatenate((irisDataset[40:50,:],
                              irisDataset[90:100,:]),axis=0)

In [14]:
#memecah dataset ke input dan label
inputTraining = dataTraining[:,0:4]
inputTesting = dataTetsing[:,0:4]
labelTraining = dataTraining[:,4]
labelTesting = dataTetsing[:, 4]

In [17]:
k=3#inputnya nilai k-Nearest Neighborsnya
#prediksi data testing menggunakan data training
matriks_prediksi_kelas = np.ndarray(shape = (0,1))
#mengkasting label ke bentuk matriks
labelTraining =np.matrix(labelTraining).T
for i in range(0,inputTesting.shape[0]): #loop semua data testing
    for j in range(0, inputTraining.shape[0]): #loop semua data training
        #hitung jarak euclidean tiap satu data testing
        #ke semua data training
        euclideanDistance = np.square(np.sum((np.tile(
            inputTesting[i,:],
            (inputTraining.shape[0],1))-inputTraining)**2, axis=1))
        #mencasting ke variabel matriks
        #ke semua data training
        euclideanDistance=np.matrix(euclideanDistance).T
        #menambahkan kolom label ke matriks euclidean
        matriksEuclideanDanLabel = np.concatenate((euclideanDistance,labelTraining), axis=1)

        #casting ke array dulu untuk disorting
        matriksEuclideanDanLabel = np.asarray(matriksEuclideanDanLabel)
        #sorting berdasarkan jarak Euclidean
        matriksEuclideanDanLabelSorted = matriksEuclideanDanLabel[
            matriksEuclideanDanLabel[:, 0].argsort()]
        #mengambil k-label kelas dengan jarak euclidean plg kecil
        k_label =matriksEuclideanDanLabelSorted[0:k,1]
        #prediksi adalah kelas dengan kemunculan terbanyak dari k-label
        prediksi_kelas = np.matrix(stats.mode(k_label)[0])
    #menggabungkan semua prediksi dalam matriks
    matriks_prediksi_kelas=np.concatenate((matriks_prediksi_kelas, 
                                           prediksi_kelas), axis=0)

In [18]:
#menghitung akurasi
matriks_prediksi_kelas=matriks_prediksi_kelas.ravel()#flatten ke 1D array
prediksiBenar = (matriks_prediksi_kelas == labelTesting).sum()
prediksiSalah = (matriks_prediksi_kelas != labelTesting).sum()
print("prediksi benar: ", prediksiBenar, "data")
print("prediksi salah: ", prediksiSalah, "data")
print("akurasi: ", prediksiBenar/(prediksiBenar+prediksiSalah) * 100, "%")

prediksi benar:  20 data
prediksi salah:  0 data
akurasi:  100.0 %
